In [1]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np
from collections import deque

In [3]:
MAX_FRAMES_LEN = 300
MAX_FRAMES_NUM = 298

# MediaPipe Pose 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)
mp_drawing = mp.solutions.drawing_utils

# 중요 관절 인덱스
important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

window = deque(maxlen = ((MAX_FRAMES_LEN // 3) * len(important_landmarks)))

test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for landmark in important_landmarks:
        test_df.loc[len(test_df)] = [frame, landmark, np.nan, np.nan, np.nan]

# 웹캠에서 영상 받기
cap = cv2.VideoCapture(0)  # 기본 카메라 사용

frame_count = 0
processed_frame = 1
start_time = time.time()
predict_time = start_time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    current_time = time.time()  
    elapsed = current_time - start_time
    predict_elapsed = current_time - predict_time

    if elapsed >= 0.1:
        start_time = current_time
        # MediaPipe에 넣기 위해 RGB로 변환
        frame = cv2.resize(frame, (480, 360))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # 관절 데이터 추출 및 표시
        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                window.append((landmark.x, landmark.y, landmark.z))

        cv2.imshow('Real-Time Pose Landmarks', frame)

        # Q 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    elif predict_elapsed >= 5:
        test_df2 = test_df.copy(deep = True)

        j = 1

        for i in range(len(window)):
            test_df2.loc[j - 1, ['x', 'y', 'z']] = [window[i][0], window[i][1], window[i][2]]
            j += 1

            if j % 13 == 1:
                j += 26

        test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)
        test_df2 = test_df2.sort_values(by = ["frame", "landmark_id"]).reset_index(drop = True)

        # frame 오름차순, source_index 우선 정렬
        test_df2 = test_df2.sort_values(by=["frame"])

        # 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
        df_pivot = test_df2.pivot(index=["frame"], columns="landmark_id", values=["x", "y", "z"])

        # 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
        df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

        # 인덱스 복구
        df_pivot.reset_index(inplace=True)
                            
        # 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
        df_pivot = df_pivot.sort_values(by=["frame"]).reset_index(drop=True)

        print(df_pivot)
        predict_time = current_time


cap.release()
cv2.destroyAllWindows()


C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\773414370.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


     frame       x_0      x_11      x_12      x_13      x_14      x_15  \
0      1.0  0.560152  0.762866  0.311928  0.826555  0.263615  0.828409   
1      2.0  0.561900  0.760757  0.312367  0.811738  0.263608  0.820263   
2      3.0  0.563648  0.758648  0.312806  0.796921  0.263601  0.812117   
3      4.0  0.565396  0.756539  0.313245  0.782103  0.263594  0.803972   
4      5.0  0.565795  0.755071  0.314229  0.777768  0.263819  0.801821   
..     ...       ...       ...       ...       ...       ...       ...   
295  296.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   
296  297.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   
297  298.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   
298  299.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   
299  300.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   

         x_16      x_23      x_24  ...      z_13      z_14      z_15  \
0    0.259859  0.674588  0.394054  ... 

C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\773414370.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


     frame       x_0      x_11      x_12      x_13      x_14      x_15  \
0      1.0  0.560152  0.762866  0.311928  0.826555  0.263615  0.828409   
1      2.0  0.561900  0.760757  0.312367  0.811738  0.263608  0.820263   
2      3.0  0.563648  0.758648  0.312806  0.796921  0.263601  0.812117   
3      4.0  0.565396  0.756539  0.313245  0.782103  0.263594  0.803972   
4      5.0  0.565795  0.755071  0.314229  0.777768  0.263819  0.801821   
..     ...       ...       ...       ...       ...       ...       ...   
295  296.0  0.548136  0.718977  0.333722  0.793598  0.258246  0.799973   
296  297.0  0.548136  0.718977  0.333722  0.793598  0.258246  0.799973   
297  298.0  0.548136  0.718977  0.333722  0.793598  0.258246  0.799973   
298  299.0  0.548136  0.718977  0.333722  0.793598  0.258246  0.799973   
299  300.0  0.548136  0.718977  0.333722  0.793598  0.258246  0.799973   

         x_16      x_23      x_24  ...      z_13      z_14      z_15  \
0    0.259859  0.674588  0.394054  ... 

C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\773414370.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


     frame       x_0      x_11      x_12      x_13      x_14      x_15  \
0      1.0  0.569514  0.749996  0.321452  0.767487  0.272736  0.793717   
1      2.0  0.569512  0.750197  0.321453  0.767879  0.272713  0.793942   
2      3.0  0.569509  0.750397  0.321454  0.768271  0.272691  0.794167   
3      4.0  0.569506  0.750598  0.321455  0.768664  0.272668  0.794392   
4      5.0  0.569264  0.750620  0.322019  0.768918  0.272627  0.794550   
..     ...       ...       ...       ...       ...       ...       ...   
295  296.0  0.420859  0.540261  0.169674  0.594388  0.102992  0.635300   
296  297.0  0.420948  0.540383  0.169699  0.594783  0.102951  0.636378   
297  298.0  0.421037  0.540505  0.169724  0.595178  0.102910  0.637457   
298  299.0  0.421037  0.540505  0.169724  0.595178  0.102910  0.637457   
299  300.0  0.421037  0.540505  0.169724  0.595178  0.102910  0.637457   

         x_16      x_23      x_24  ...      z_13      z_14      z_15  \
0    0.263956  0.683791  0.417702  ... 

C:\Users\이민하\AppData\Local\Temp\ipykernel_16300\773414370.py:83: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df2 = test_df2.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop = True)


     frame       x_0      x_11      x_12      x_13      x_14      x_15  \
0      1.0  0.500252  0.740819  0.332643  0.988596  0.263597  0.723186   
1      2.0  0.500597  0.740501  0.334103  0.988270  0.262703  0.718708   
2      3.0  0.500942  0.740182  0.335564  0.987945  0.261809  0.714230   
3      4.0  0.501286  0.739864  0.337024  0.987619  0.260916  0.709751   
4      5.0  0.504008  0.739187  0.336479  0.987609  0.260238  0.708852   
..     ...       ...       ...       ...       ...       ...       ...   
295  296.0  0.428818  0.539075  0.175338  0.573281  0.091772  0.595636   
296  297.0  0.427507  0.539865  0.173716  0.575268  0.090396  0.596021   
297  298.0  0.426196  0.540655  0.172093  0.577255  0.089019  0.596406   
298  299.0  0.426196  0.540655  0.172093  0.577255  0.089019  0.596406   
299  300.0  0.426196  0.540655  0.172093  0.577255  0.089019  0.596406   

         x_16      x_23      x_24  ...      z_13      z_14      z_15  \
0    0.199555  0.683775  0.410769  ... 

In [12]:
window

deque([(0.5375720262527466, 0.3480285704135895, -0.27250340580940247),
       (0.6800273656845093, 0.5129035711288452, 0.053052034229040146),
       (0.41963377594947815, 0.5204527378082275, 0.09404278546571732),
       (0.7491519451141357, 0.7471628189086914, 0.07853985577821732),
       (0.38924869894981384, 0.7587519884109497, 0.06900093704462051),
       (0.7408665418624878, 0.8805132508277893, 0.011267958208918571),
       (0.4045853614807129, 0.8050075769424438, -0.07184842228889465),
       (0.6200060248374939, 0.9719470739364624, -0.016301948577165604),
       (0.454659640789032, 0.9799738526344299, 0.01787189207971096),
       (0.834904134273529, 0.7970061302185059, -0.575016438961029),
       (0.2658470869064331, 0.7413482069969177, -0.49565932154655457),
       (0.5444836020469666, 1.267563819885254, -0.576308012008667),
       (0.5459272265434265, 1.1457544565200806, -0.5263139605522156),
       (0.5377752780914307, 0.3764154314994812, -0.3390568792819977),
       (0.670164

In [11]:
test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for landmark in important_landmarks:
        test_df.loc[len(test_df)] = [frame, landmark, np.nan, np.nan, np.nan]

test_df

,frame,landmark_id,x,y,z
0,1.0,0.0,NaN,NaN,NaN
1,1.0,11.0,NaN,NaN,NaN
2,1.0,12.0,NaN,NaN,NaN
3,1.0,13.0,NaN,NaN,NaN
4,1.0,14.0,NaN,NaN,NaN
...,...,...,...,...,...
3895,300.0,24.0,NaN,NaN,NaN
3896,300.0,25.0,NaN,NaN,NaN
3897,300.0,26.0,NaN,NaN,NaN
3898,300.0,29.0,NaN,NaN,NaN


In [ ]:
# 0 ~ 12, 13 ~ 25, 26 ~ 38, 39 ~ 51, 52 ~ 63, 64 ~ 76, 77 ~ 89

In [17]:
j = 1

for i in range(len(window)):
    test_df.loc[j - 1][['x', 'y', 'z']] = [window[i][0], window[i][1], window[i][2]]
    j += 1

    if j % 13 == 1:
        j += 26

test_df


C:\Users\이민하\AppData\Local\Temp\ipykernel_21132\1634713654.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_df.loc[j - 1][['x', 'y', 'z']] = [window[i][0], window[i][1], window[i][2]]
C:\Users\이민하\AppData\Local\Temp\ipykernel_21132\1

,frame,landmark_id,x,y,z
0,1.0,0.0,0.537572,0.348029,-0.272503
1,1.0,11.0,0.680027,0.512904,0.053052
2,1.0,12.0,0.419634,0.520453,0.094043
3,1.0,13.0,0.749152,0.747163,0.078540
4,1.0,14.0,0.389249,0.758752,0.069001
...,...,...,...,...,...
3895,300.0,24.0,NaN,NaN,NaN
3896,300.0,25.0,NaN,NaN,NaN
3897,300.0,26.0,NaN,NaN,NaN
3898,300.0,29.0,NaN,NaN,NaN


In [26]:
test_df[test_df["frame"] == 7]

,frame,landmark_id,x,y,z
78,7.0,0.0,0.539048,0.415920,-0.504662
79,7.0,11.0,0.668463,0.526017,-0.192750
80,7.0,12.0,0.421224,0.546063,-0.152645
81,7.0,13.0,0.759446,0.721382,-0.086590
82,7.0,14.0,0.387371,0.754073,-0.042249
83,7.0,15.0,0.691124,0.810436,-0.062729
84,7.0,16.0,0.392159,0.827733,-0.086182
85,7.0,23.0,0.616072,0.952706,-0.023893
86,7.0,24.0,0.460314,0.942816,0.025026
87,7.0,25.0,0.810715,0.866585,-0.528213


In [50]:
test_df = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

for frame in np.arange(1, 301):
    for idx in important_landmarks:
        test_df.loc[len(test_df)] = [frame, idx, np.nan, np.nan, np.nan]

test_df

,frame,landmark_id,x,y,z
0,1.0,0.0,NaN,NaN,NaN
1,1.0,11.0,NaN,NaN,NaN
2,1.0,12.0,NaN,NaN,NaN
3,1.0,13.0,NaN,NaN,NaN
4,1.0,14.0,NaN,NaN,NaN
...,...,...,...,...,...
3895,300.0,24.0,NaN,NaN,NaN
3896,300.0,25.0,NaN,NaN,NaN
3897,300.0,26.0,NaN,NaN,NaN
3898,300.0,29.0,NaN,NaN,NaN


In [ ]:
test_df = test_df.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
test_df = test_df.sort_values(by = ["frame", "landmark_id"])

SyntaxError: invalid syntax (2259856835.py, line 1)